[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/csharp/Samples)


In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget:Microsoft.Data.Analysis,0.19.0"
#r "nuget:XPlot.Plotly.Interactive, 4.0.6"

using Microsoft.Data.Analysis;

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [ ]:
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<int> ints = new PrimitiveDataFrameColumn<int>("Ints", 3); // Makes a column of length 3. Filled with nulls initially
StringDataFrameColumn strings = new StringDataFrameColumn("Strings", 3); // Makes a column of length 3. Filled with nulls initially

In [ ]:
// Append 3 values to dateTimes
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/02"));

In [ ]:
DataFrame df = new DataFrame(dateTimes, ints, strings ); // This will throw if the columns are of different lengths

In [ ]:
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,<null>,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
// To change a value directly through df
df[0, 1] = 10; // 0 is the rowIndex, and 1 is the columnIndex. This sets the 0th value in the Ints columns to 10
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,<null>,<null>
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
// Modify ints and strings columns by indexing
ints[1] = 100;
strings[1] = "Foo!";
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,10,<null>
1,2019-01-01 00:00:00Z,100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
// Indexing can throw when types don't match.
// ints[1] = "this will throw because I am a string";  
// Info can be used to figure out the type of data in a column. 
df.Info()

index,Info,DateTimes,Ints,Strings
0,DataType,System.DateTime,System.Int32,System.String
1,Length (excluding null values),3,2,3


In [ ]:
// Add 5 to ints through the DataFrame
df["Ints"].Add(5, inPlace: true);
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,15,<null>
1,2019-01-01 00:00:00Z,105,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
// We can also use binary operators. Binary operators produce a copy, so assign it back to our Ints column 
df["Ints"] = (ints / 5) * 100;
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,300,<null>
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,<null>,<null>


In [ ]:
// Fill nulls in our columns, if any. Ints[2], Strings[0] and Strings[1] are null
df["Ints"].FillNulls(-1, inPlace: true);
df["Strings"].FillNulls("Bar", inPlace: true);
df

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-01 00:00:00Z,2100,Foo!
2,2019-01-02 00:00:00Z,-1,Bar


In [ ]:
// To inspect the first row
DataFrameRow row0 = df.Rows[0];
row0

index,type,value
0,System.DateTime,2019-01-01 00:00:00Z
1,System.Int32,300
2,System.String,Bar


In [ ]:
// Filter rows based on equality
PrimitiveDataFrameColumn<bool> boolFilter = df["Strings"].ElementwiseEquals("Bar");
boolFilter

index,value
0,True
1,False
2,True


In [ ]:
DataFrame filtered = df.Filter(boolFilter);
filtered

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,300,Bar
1,2019-01-02 00:00:00Z,-1,Bar


In [ ]:
// Sort our dataframe using the Ints column
DataFrame sorted = df.OrderBy("Ints");
sorted

index,DateTimes,Ints,Strings
0,2019-01-02 00:00:00Z,-1,Bar
1,2019-01-01 00:00:00Z,300,Bar
2,2019-01-01 00:00:00Z,2100,Foo!


In [ ]:
// GroupBy 
GroupBy groupBy = df.GroupBy("DateTimes");
// Count of values in each group
DataFrame groupCounts = groupBy.Count();
groupCounts

index,DateTimes,Ints,Strings
0,2019-01-01 00:00:00Z,2,2
1,2019-01-02 00:00:00Z,1,1


In [ ]:
// Alternatively find the sum of the values in each group in Ints
DataFrame intsGroupSum = groupBy.Sum("Ints");
intsGroupSum

index,DateTimes,Ints
0,2019-01-01 00:00:00Z,2400
1,2019-01-02 00:00:00Z,-1


In [ ]:
using XPlot.Plotly;
using System.Linq;

In [ ]:
#r "nuget:MathNet.Numerics,4.9.0"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages MathNet.Numerics, 4.9.0

In [ ]:
using MathNet.Numerics.Distributions;
double mean = 0;
double stdDev = 0.1;

MathNet.Numerics.Distributions.Normal normalDist = new Normal(mean, stdDev);

In [ ]:
PrimitiveDataFrameColumn<double> doubles = new PrimitiveDataFrameColumn<double>("Normal Distribution", normalDist.Samples().Take(1000));
display(Chart.Plot(
    new Histogram()
    {
        x = doubles,
        nbinsx = 30
    }
));

<!DOCTYPE html>